In [5]:
!pip install tensorflow
!pip install matplotlib


/Users/shingcholi/anaconda3/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
python(5066) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(5067) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [12]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

mixed_folder_path = "train"

CLASS_NAMES = ["Aluminum", "Paper", "Plastic", "Unrecycled"]

def parse_voc_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    filename = root.find("filename").text
    label = root.find("object").find("name").text
    label_index = CLASS_NAMES.index(label) if label in CLASS_NAMES else None
    
    return filename, label_index

image_files, labels = [], []
for file in os.listdir(mixed_folder_path):
    if file.endswith(".xml"):
        xml_file = os.path.join(mixed_folder_path, file)
        filename, label_index = parse_voc_xml(xml_file)
        if label_index is not None:
            image_file_path = os.path.join(mixed_folder_path, filename)
            if os.path.exists(image_file_path):
                image_files.append(image_file_path)
                labels.append(label_index)

print("Loaded images:", image_files[:5])
print("Loaded labels:", labels[:5])


Loaded images: ['train/not-recylced-2_jpg.rf.8ab9d65ae4f7d5f5a516183296cd01d9.jpg', 'train/can-3_jpg.rf.31df59620fa5ee3e452d542b816c4e00.jpg', 'train/plastic-1_jpg.rf.0bce7acfc2d32d5aaf3190e6a732e7ae.jpg', 'train/plastic-4_png.rf.50c1b8715afca0a7702b5faa3cd9fe9b.jpg', 'train/tissue-1_jpg.rf.af27687ed4bd02b5574ca7172668b035.jpg']
Loaded labels: [1, 0, 2, 2, 1]


In [13]:
def preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.mobilenet.preprocess_input(image)
    return image, label

dataset = tf.data.Dataset.from_tensor_slices((image_files, labels))
dataset = dataset.map(preprocess_image)

train_size = int(0.8 * len(image_files))
train_dataset = dataset.take(train_size).batch(32).shuffle(buffer_size=1000)
val_dataset = dataset.skip(train_size).batch(32)


In [14]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(CLASS_NAMES), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.9634 - val_accuracy: 0.5000 - val_loss: 0.7968
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 1.0000 - loss: 0.1522 - val_accuracy: 1.0000 - val_loss: 0.3113
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 1.0000 - loss: 0.0271 - val_accuracy: 1.0000 - val_loss: 0.1897
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 1.0000 - val_loss: 0.1203
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 0.0799
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 1.0000 - loss: 8.7732e-04 - val_accuracy: 1.0000 - val_loss: 0.0582
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 1.0000 - loss: 2.8313e-04 - val_accuracy: 1.0000 - val_loss: 0.0473
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 1.0000 - loss: 1.0901e-04 - val_accuracy: 1.000

In [18]:
model.save('generalWaste.h5')

In [20]:
model = tf.keras.models.load_model('generalWaste.h5')